# Blender Parameterization Helper Testing


## Imports & Setup



## Load & Parse Code



### Import Test Code


In [728]:

test_code = """
###############
### Imports ###
###############

import bpy
import math


############
### Prep ###
############

# Clear Scene
bpy.ops.object.select_all(action='DESELECT')
bpy.ops.object.select_by_type(type='MESH')
bpy.ops.object.delete()

##################
### Parameters ###
##################

weave_name = "Orbital Vipera Beerus Lace"

large_ring_minor = 0.18
large_ring_major = 1.42

small_ring_minor = 0.17
small_ring_major = 1.00

dist_1 = 1.50
dist_2 = 0.30
dist_3 = 1.20

num_units = 12

angle_1 = math.radians(73)
angle_2 = math.radians(164)
angle_3 = math.radians(90)


##################
### Model Code ###
##################

# Initial Small Ring
bpy.ops.mesh.primitive_torus_add(
    align="WORLD",
    location=(0,0,0),
    rotation=(angle_1,0,0),
    major_radius=small_ring_major,
    minor_radius=small_ring_minor,
    major_segments=48,
    minor_segments=12,
)

for i in range(num_units):
    base_dist = dist_1 * (i+1)
    
    mod = -1 if i % 2 == 0 else 1

    # Vipera Small Ring
    bpy.ops.mesh.primitive_torus_add(
        align="WORLD",
        location=(base_dist,0,0),
        rotation=(mod * angle_1,0,0),
        major_radius=small_ring_major,
        minor_radius=small_ring_minor,
        major_segments=48,
        minor_segments=12,
    )

    # Lace Small Ring
    if i != 0 and i != num_units-1:
        bpy.ops.mesh.primitive_torus_add(
            align="WORLD",
            location=(base_dist,-mod * dist_3,0),
            rotation=(angle_3,0,0),
            major_radius=small_ring_major,
            minor_radius=small_ring_minor,
            major_segments=48,
            minor_segments=12,
        )

    # Large Ring
    bpy.ops.mesh.primitive_torus_add(
        align="WORLD",
        location=(base_dist,mod * dist_2,0),
        rotation=(0,angle_2,0),
        major_radius=large_ring_major,
        minor_radius=large_ring_minor,
        major_segments=48,
        minor_segments=12,
    )

# Final Small Ring
mod = -1 if num_units % 2 == 0 else 1
base_dist = dist_1 * (num_units + 1)
bpy.ops.mesh.primitive_torus_add(
    align="WORLD",
    location=(base_dist,0,0),
    rotation=(mod * angle_1,0,0),
    major_radius=small_ring_major,
    minor_radius=small_ring_minor,
    major_segments=48,
    minor_segments=12,
)
"""



### Split Code By Section


In [729]:

# Set up section containers
section_mapping = {
    "Imports":   "",
    "Parameters": "",
    "Model Code": "",
}

# Initialize section tracking variable
current_section = None

# Itterate over lines to get sections of interest
for line in test_code.split('\n'):

    # Check for section headers
    if ("### " in line) and (" ###" in line):
        current_section = line.replace("#","").strip()

    # Itterate over lines in a target section
    if current_section in section_mapping:

        # Skip useless lines
        if line.strip() == "": continue # empty lines
        if line.startswith("#"): continue # comments

        # Add line to the current section
        section_mapping[current_section] += line + "\n"



### Parse Parameters


In [730]:

parameters = dict()

# Parse parameters section lines for name
for line in section_mapping["Parameters"].split('\n'):
    if "=" in line:

        # Get Values from lines
        split_vals = line.split("=")
        name, value = map(str.strip, split_vals)

        # Check type of value
        value_type = None
        if '"' in  value or "'" in value:
            value_type = "str"
        elif value.replace('.', '', 1).isdigit():
            value_type = "number"
        elif "radians" in value:
            value_type = "angle"
            value = value.replace("radians(", "").replace(")", "").replace("math.", "").strip()
            value = f"{value}/180 * math.pi"  # Convert to radians
        else:
            value_type = "Unknown"

        parameters[name] = [value, value_type]



## Create Output Code



### Initialize Output Variable


In [731]:

output = """
###############
### Imports ###
###############

"""



### Add Imports to Output


In [732]:

# Check for bpy import
imports_bpy = False
for line in section_mapping["Imports"].split('\n'):
    if line.strip() == "import bpy":
        imports_bpy = True

# Import bpy if not present
if not imports_bpy:
    section_mapping["Imports"] = "import bpy\n" + section_mapping["Imports"]

# Check for import of math library
imports_math = False
for line in section_mapping["Imports"].split('\n'):
    if line.strip() == "import math":
        imports_math = True

# Import math if not present
if not imports_math:
    section_mapping["Imports"] = "import math\n" + section_mapping["Imports"]

output += section_mapping["Imports"] + "\n\n"



### Set Weave Name if Provided


In [733]:

weave_name = parameters.get("weave_name")
if weave_name is not None:
    weave_name = weave_name[0]
    output += "################\n### Settings ###\n################\n\n# Weave Name\n"
    output += f"weave_name = {weave_name}\n\n\n"



### Create Update Model Function using Code Section


In [734]:

# Add section header
output += "#############################\n### Model Update Function ###\n#############################\n\n"

# Add function definition
output += "def update_model(self, context):\n\n"

# Add clean up section
output += "\t################\n\t### Clean Up ###\n\t################\n\n"
output += "\t# Clear Scene - Delete all Objects\n"
output += "\tbpy.ops.object.select_all(action='DESELECT')\n"
output += "\tbpy.ops.object.select_by_type(type='MESH')\n"
output += "\tbpy.ops.object.delete()\n\n"

# Add Code  header
output += "\t####################\n\t### Create Rings ###\n\t####################\n\n"

# Replace parameters in model code with self.<parameter_name>
model_code = section_mapping["Model Code"]
for key in parameters:
    model_code = model_code.replace(key, f"self.{key}")

# Clean up cases where "self.self." could appear (if one parameter name is a substring of another e.g. "angle_1" and "angle_10")
while "self.self." in model_code:
    model_code = model_code.replace("self.self.", "self.")

# Prepend tabs and add to output
for line in model_code.split('\n'):
    output += "\t" + line + "\n"



### Define Custom Menu Contents


In [735]:

# Add Section Heading
output += "\n\n############################\n### Define Menu Contents ###\n############################\n\n"

# Add class definition
output += "class ChainmailProperties(bpy.types.PropertyGroup):\n"

# Add properties for each parameter
for key, value in parameters.items():
    if key != "weave_name":

        # Create clean name for display
        clean_name = key.replace("_", " ").capitalize()

        # Create property
        output += f"\t{key}: bpy.props.FloatProperty(\n"
        output += f'\t\tname="{clean_name}",\n'
        output += f'\t\tdefault={value[0]},\n'

        # Add specific type handling
        if value[1] == "angle":
            output += '\t\tsubtype="ANGLE",\n'
            output += '\t\tunit="ROTATION",\n'

        # Close out property definition
        output += '\t\tupdate=update_chainmail\n'
        output += "\t)\n"



### Create Custom Menu & Add Contents


In [736]:

# Add Section Heading
output += "\n\n###################\n### Create Menu ###\n###################\n\n"

# Add class definition
output += "class VIEW3D_PT_chainmail_panel(bpy.types.Panel):\n"

# Add weave name
if "weave_name" in parameters:
    weave_name = parameters["weave_name"][0].replace('"', '').replace("'", "")
    output += f'\tbl_label = "{weave_name} Modification Pannel"\n'
else:
    output += '\tbl_label = "Chainmail Modification Pannel"\n'

# Finish class variables
output += "\tbl_idname = 'VIEW3D_PT_chainmail_ring'\n\tbl_space_type = 'VIEW_3D'\n\tbl_region_type = 'UI'\n\tbl_category = 'Chainmail'\n\n"

# Add definition of draw function
output += "\tdef draw(self, context):\n"

# Add layout & props
output += "\t\tlayout = self.layout\n"
output += "\t\tprops = context.scene.chainmail_props\n\n"

# Add item for each parameter
for key in parameters:
    if key != "weave_name":
        output += f'\t\tlayout.prop(props, "{key}")\n'

# Add button to start keyboard controls
output += "\n\t\tlayout.separator()\n\t\tlayout.operator('wm.chainmail_tweak_modal', text='Keyboard Tweak Mode')\n"



### Create Keyboard Controls


In [ ]:

# Add Section Heading
output += "\n\n################################\n### Create Keyboard Controls ###\n################################\n\n"

# Add global variable to track selected index
output += "tweaker_selected_index = 0\n"

# Add Class Definition
output += "class ChainmailTweaker(bpy.types.Operator):\n\n"

output += '\tbl_idname = "wm.chainmail_tweak_modal"\n\tbl_label  = "Chainmail Tweaker (Arrow Keys + Tab)"\n\n'

# Add variable names
output += "\tvar_names = [\n"
for key in parameters:
    if key != "weave_name":
        output += f'\t\t"{key}",\n'
output += "\t]\n\n"


## define functions
tmp = """
    def __init__(self):
        global tweaker_selected_index
        self.selected_index = tweaker_selected_index

    def invoke(self, context, event):
        self._area = context.area
        context.window_manager.modal_handler_add(self)
        self.update_display(context)
        return {'RUNNING_MODAL'}

    def modal(self, context, event):
        global tweaker_selected_index
        
        props = context.scene.chainmail_props

        if event.type == 'ESC':
            self._area.header_text_set(None)
            return {'CANCELLED'}

        if event.type in {'MIDDLEMOUSE', 'RIGHTMOUSE', 'WHEELUPMOUSE', 'WHEELDOWNMOUSE'}:
            return {'PASS_THROUGH'}

        if event.value == 'PRESS':
            step = 0.10
            if event.shift: step = 1.00
            if event.ctrl:  step = 0.05
            if event.alt:   step = 0.01

            var_name = self.var_names[self.selected_index]

            if event.type == 'TAB':
                if event.shift:
                    self.selected_index = (self.selected_index - 1) % len(self.var_names)
                    tweaker_selected_index = self.selected_index
                    self.update_display(context)
                else:
                    self.selected_index = (self.selected_index + 1) % len(self.var_names)
                    tweaker_selected_index = self.selected_index
                    self.update_display(context)

            elif event.type == 'RIGHT_ARROW':
                self.change_value(props, var_name, step)
                self.update_display(context)

            elif event.type == 'LEFT_ARROW':
                self.change_value(props, var_name, -step)
                self.update_display(context)

        return {'RUNNING_MODAL'}

    def change_value(self, props, var_name, delta):
        current = getattr(props, var_name)
        setattr(props, var_name, current + delta)

    def update_display(self, context):
        props = context.scene.chainmail_props
        var_name = self.var_names[self.selected_index]
        value = getattr(props, var_name)
        self._area.header_text_set(f"Adjusting: {var_name} = {value:.3f}  (←/→ to adjust, Tab to switch, Esc to exit)")

"""

# Add function defintions to the outpout
output += tmp.replace("    ", "\t")





## Display Updated Code


In [738]:

print(output)



###############
### Imports ###
###############

import bpy
import math


################
### Settings ###
################

# Weave Name
weave_name = "Orbital Vipera Beerus Lace"


#############################
### Model Update Function ###
#############################

def update_model(self, context):

	################
	### Clean Up ###
	################

	# Clear Scene - Delete all Objects
	bpy.ops.object.select_all(action='DESELECT')
	bpy.ops.object.select_by_type(type='MESH')
	bpy.ops.object.delete()

	####################
	### Create Rings ###
	####################

	bpy.ops.mesh.primitive_torus_add(
	    align="WORLD",
	    location=(0,0,0),
	    rotation=(self.angle_1,0,0),
	    major_radius=self.small_ring_major,
	    minor_radius=self.small_ring_minor,
	    major_segments=48,
	    minor_segments=12,
	)
	for i in range(self.num_units):
	    base_dist = self.dist_1 * (i+1)
	    mod = -1 if i % 2 == 0 else 1
	    # Vipera Small Ring
	    bpy.ops.mesh.primitive_torus_add(
	  